This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-features/model_hot_swap).

## Model Hot Swap Tutorial

One of the biggest challenges facing organizations once they have a model trained is deploying the model:  Getting all of the resources together, MLOps configured and systems prepared to allow inferences to run.

The next biggest challenge?  Replacing the model while keeping the existing production systems running.

This tutorial demonstrates how Wallaroo model hot swap can update a pipeline step with a new model with one command.  This lets organizations keep their production systems running while changing a ML model, with the change taking only milliseconds, and any inference requests in that time are processed after the hot swap is completed.

This example and sample data comes from the Machine Learning Group's demonstration on [Credit Card Fraud detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).

This tutorial provides the following:

* `ccfraud.onnx`: A pre-trained ML model used to detect potential credit card fraud.
* `xgboost_ccfraud.onnx`: A pre-trained ML model used to detect potential credit card fraud originally converted from an XGBoost model.  This will be used to swap with the `ccfraud.onnx`.
* `smoke_test.json`: A data file used to verify that the model will return a low possibility of credit card fraud.
* `high_fraud.json`: A data file used to verify that the model will return a high possibility of credit card fraud.
* Sample inference data files: Data files used for inference examples with the following number of records:
  * `cc_data_5.json`: 5 records.
  * `cc_data_1k.json`: 1,000 records.
  * `cc_data_10k.json`: 10,000 records.
  * `cc_data_40k.json`: Over 40,000 records.

## Reference

For more information about Wallaroo and related features, see the [Wallaroo Documentation Site](https://docs.wallaroo.ai).

## Steps

The following steps demonstrate the following:

* Connect to a Wallaroo instance.
* Create a workspace and pipeline.
* Upload both models to the workspace.
* Deploy the pipe with the `ccfraud.onnx` model as a pipeline step.
* Perform sample inferences.
* Hot swap and replace the existing model with the `xgboost_ccfraud.onnx` while keeping the pipeline deployed.
* Conduct additional inferences to demonstrate the model hot swap was successful.
* Undeploy the pipeline and return the resources back to the Wallaroo instance.

### Load the Libraries

Load the Python libraries used to connect and interact with the Wallaroo instance.

In [7]:
import wallaroo
from wallaroo.object import EntityNotFoundError

### Open a Connection to Wallaroo

The first step is to connect to Wallaroo through the Wallaroo client.

This is accomplished using the `wallaroo.Client(api_endpoint, auth_endpoint, auth_type command)` command that connects to the Wallaroo instance services.

The `Client` method takes the following parameters:

* **api_endpoint** (*String*): The URL to the Wallaroo instance API service.
* **auth_endpoint** (*String*): The URL to the Wallaroo instance Keycloak service.
* **auth_type command** (*String*): The authorization type.  In this case, `SSO`.

The URLs are based on the Wallaroo Prefix and Wallaroo Suffix for the Wallaroo instance.  For more information, see the [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).  In the example below, replace "YOUR PREFIX" and "YOUR SUFFIX" with the Wallaroo Prefix and Suffix, respectively.

If connecting from within the Wallaroo instance's JupyterHub service, then only `wl = wallaroo.Client()` is required.

Once run, the `wallaroo.Client` command provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Depending on the configuration of the Wallaroo instance, the user will either be presented with a login request to the Wallaroo instance or be authenticated through a broker such as Google, Github, etc.  To use the broker, select it from the list under the username/password login forms.  For more information on Wallaroo authentication configurations, see the [Wallaroo Authentication Configuration Guides](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-sso-authentication/).

In [8]:
# Remote Login

wallarooPrefix = "YOUR PREFIX"
wallarooSuffix = "YOUR SUFFIX"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

# Internal Login

# wl = wallaroo.Client()

### Set the Variables

The following variables are used in the later steps for creating the workspace, pipeline, and uploading the models.  Modify them according to your organization's requirements.

In [9]:
workspace_name = 'hotswapworkspace'
pipeline_name = 'hotswappipeline'
original_model_name = 'ccfraudoriginal'
original_model_file_name = './ccfraud.onnx'
replacement_model_name = 'ccfraudreplacement'
replacement_model_file_name = './xgboost_ccfraud.onnx'

In [10]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

### Create the Workspace

We will create a workspace based on the variable names set above, and set the new workspace as the `current` workspace.  This workspace is where new pipelines will be created in and store uploaded models for this session.

Once set, the pipeline will be created.

In [11]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)
pipeline

name,hotswappipeline
created,2023-01-18 16:44:54.245465+00:00
last_updated,2023-01-18 16:44:54.245465+00:00
deployed,(none)
tags,
versions,9c993849-94d0-4f54-9fbe-9d62d2833cb3
steps,


### Upload Models

We can now upload both of the models.  In a later step, only one model will be added as a [pipeline step](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipeline/#add-a-step-to-a-pipeline), where the pipeline will submit inference requests to the pipeline.

In [12]:
original_model = wl.upload_model(original_model_name , original_model_file_name)
replacement_model = wl.upload_model(replacement_model_name , replacement_model_file_name)

### Add Model to Pipeline Step

With the models uploaded, we will add the original model as a pipeline step, then deploy the pipeline so it is available for performing inferences.

In [13]:
pipeline.add_model_step(original_model)
pipeline

name,hotswappipeline
created,2023-01-18 16:44:54.245465+00:00
last_updated,2023-01-18 16:44:54.245465+00:00
deployed,(none)
tags,
versions,9c993849-94d0-4f54-9fbe-9d62d2833cb3
steps,


In [23]:
pipeline.deploy()

name,hotswappipeline
created,2023-01-18 16:44:54.245465+00:00
last_updated,2023-01-18 16:55:46.047515+00:00
deployed,True
tags,
versions,"d0295b33-f1f9-420d-a1a6-c55485a13715, 9ccee697-98e5-4635-8db6-6ff8e577a47e, ab55afb4-2473-466b-b6eb-594f1a5727c3, 9c993849-94d0-4f54-9fbe-9d62d2833cb3"
steps,ccfraudoriginal


In [24]:
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.2.153',
   'name': 'engine-96486c95d-zfchr',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'hotswappipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'ccfraudreplacement',
      'version': '714efd19-5c83-42a8-aece-24b4ba530925',
      'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.2.154',
   'name': 'engine-lb-55dcdff64c-9np9k',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

### Verify the Model

The pipeline is deployed with our model.  The following will verify that the model is operating correctly.  The `high_fraud.json` file contains data that the model should process as a high likelihood of being a fraudulent transaction.

In [16]:
pipeline.infer_from_file('./high_fraud.json')

[InferenceResult({'check_failures': [],
  'elapsed': 123002,
  'model_name': 'ccfraudoriginal',
  'model_version': '3a03dc94-716e-46bb-84c8-91bc99ceb2c3',
  'original_data': {'tensor': [[1.0678324729342086,
                                18.155556397512136,
                                -1.658955105843852,
                                5.2111788045436445,
                                2.345247064454334,
                                10.467083577773014,
                                5.0925820522419745,
                                12.82951536371218,
                                4.953677046849403,
                                2.3934736228338225,
                                23.912131817957253,
                                1.7599568310350209,
                                0.8561037518143335,
                                1.1656456468728569,
                                0.5395988813934498,
                                0.7784221343010385,
                

### Replace the Model

The pipeline is currently deployed and is able to handle inferences.  The model will now be replaced without having to undeploy the pipeline.  This is done using the pipeline method [`replace_with_model_step(index, model)`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-reference-guide/pipeline/#Pipeline.replace_with_model_step).  Steps start at `0`, so the method called below will replace step 0 in our pipeline with the replacement model.

As an exercise, this deployment can be performed while inferences are actively being submitted to the pipeline to show how quickly the swap takes place.

In [19]:
pipeline.replace_with_model_step(0, replacement_model).deploy()

name,hotswappipeline
created,2023-01-18 16:44:54.245465+00:00
last_updated,2023-01-18 16:47:12.577511+00:00
deployed,True
tags,
versions,"9ccee697-98e5-4635-8db6-6ff8e577a47e, ab55afb4-2473-466b-b6eb-594f1a5727c3, 9c993849-94d0-4f54-9fbe-9d62d2833cb3"
steps,ccfraudoriginal


### Verify the Swap

To verify the swap, we'll submit a set of inferences to the pipeline using the new model.

In [20]:
pipeline.infer_from_file('./cc_data_5.json')

[InferenceResult({'check_failures': [],
  'elapsed': 121101,
  'model_name': 'ccfraudreplacement',
  'model_version': '3a03dc94-716e-46bb-84c8-91bc99ceb2c3',
  'original_data': {'tensor': [[-1.060329750089797,
                                2.354496709462385,
                                -3.563878832646437,
                                5.138734892618555,
                                -1.23084570186641,
                                -0.7687824607744093,
                                -3.588122810891446,
                                1.888083766259287,
                                -3.2789674273886593,
                                -3.956325455353324,
                                4.099343911805088,
                                -5.653917639476211,
                                -0.8775733373342495,
                                -9.131571191990632,
                                -0.6093537872620682,
                                -3.748027677256424,
           

### Undeploy the Pipeline

With the tutorial complete, the pipeline is undeployed to return the resources back to the Wallaroo instance.

In [22]:
pipeline.undeploy()

name,hotswappipeline
created,2023-01-18 16:44:54.245465+00:00
last_updated,2023-01-18 16:47:12.577511+00:00
deployed,False
tags,
versions,"9ccee697-98e5-4635-8db6-6ff8e577a47e, ab55afb4-2473-466b-b6eb-594f1a5727c3, 9c993849-94d0-4f54-9fbe-9d62d2833cb3"
steps,ccfraudoriginal
